In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel("../data/chargeback_data.xlsx", sheet_name=0)
df.columns= df.columns.str.lower()
df.rename(columns={"dia":"data", "hora":"hora_completa"}, inplace=True)
df

,data,hora_completa,valor,cartão,cbk
0,2015-05-01 00:00:00,00:01:54,36.54,536518******2108,Não
1,2015-05-01 00:00:00,00:03:46,36.54,536518******2108,Não
2,2015-05-01 00:00:00,00:08:50,69,453211******1239,Não
3,2015-05-01 00:00:00,00:27:00,193.43,548827******1705,Não
4,2015-05-01 00:00:00,01:32:46,132,531681******9778,Não
...,...,...,...,...,...
11123,2015-05-30 23:07:01.000,53,514868******7409,Não,NaN
11124,2015-05-30 23:08:47.000,15,439354******5281,Não,NaN
11125,2015-05-30 23:15:24.000,20,549167******1648,Não,NaN
11126,2015-05-30 23:17:41.000,70,518759******8384,Não,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11128 entries, 0 to 11127
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   data           11128 non-null  object
 1   hora_completa  11128 non-null  object
 2   valor          11128 non-null  object
 3   cartão         11128 non-null  object
 4   cbk            11004 non-null  object
dtypes: object(5)
memory usage: 434.8+ KB


### Missing values

In [4]:
# Checando missing values
df.isna().sum()

data               0
hora_completa      0
valor              0
cartão             0
cbk              124
dtype: int64

### Removing duplicates

In [5]:
df.duplicated().sum()

1

In [6]:
df[df.duplicated()]

,data,hora_completa,valor,cartão,cbk
6104,2015-05-15 00:00:00,23:00:20,264,515894******6461,Sim


In [7]:
df.drop_duplicates(inplace=True)

### Feature engineering

In [8]:
def get_day_period(time):
    period = ""
    if 6 <= time <= 12:
        period = "manha"
    elif 13 <= time <= 18:
        period = "tarde"
    elif 19 <= time <= 23:
        period = "noite"
    elif 00 <= time <= 5:
        period = "madrugada"
    return period

def get_month_period(day):
    period = ""
    if 1 <= day <= 10:
        period = "inicio"
    elif 11 <= day <= 20:
        period = "meio"
    elif 21 <= day <= 31:
        period = "fim"
    return period

def get_week_period(day):
    period = ""
    if day in range(0,5):
        period = "semana"
    else:
        period = "fds"

    return period

def get_value_mean_by_col(data, col, value):
    mean_day = data.groupby(col)['valor'].mean()
    value_day = round(mean_day[value], 2)
    return value_day

def get_count(data, col, value):
    card_count = data[col].value_counts()[value]
    return card_count

In [9]:
# Corrigindo as ultimas linhas que estavam sem a coluna de hora
first_wrong_row = df[df.cbk.isna()].index[0]
for x in range(first_wrong_row,df.index[-1] + 1):
    df.loc[x, 'hora_completa':] = df.loc[x, 'hora_completa':].shift()
df.loc[first_wrong_row:, "hora_completa"] = pd.to_datetime(df.loc[first_wrong_row:, "data"]).dt.time 
df['valor'] = df['valor'].astype(float)

In [10]:
# Extraindo apenas o dia, pois os meses e os anos são todos iguais
df.loc[:, "dia"] = pd.to_datetime(df.loc[:, "data"]).dt.day.astype(int)
df.loc[:, "periodo_mes"] = df.loc[:, "dia"].apply(lambda x: get_month_period(x))
df.loc[:, "dia_semana"] = pd.to_datetime(df.loc[:, "data"]).dt.day_of_week.astype(int)
df.loc[:, "periodo_semana"] = df.loc[:, "dia_semana"].apply(lambda x: get_week_period(x))
df.loc[:, "hora"] = df.loc[:, "hora_completa"].apply(lambda x: x.hour)
df.loc[:, "hora_completa_segundos"] = df.loc[:, "hora_completa"].apply(lambda x: (x.hour * 60 + x.minute) * 60 + x.second)
df.loc[:, "periodo_dia"] = df.loc[:, "hora"].apply(lambda x: get_day_period(x))
df.loc[:, "media_dia"] = df.loc[:, "dia"].apply(lambda x: get_value_mean_by_col(df, 'dia', x))
df.loc[:, "media_cartao"] = df.loc[:, "cartão"].apply(lambda x: get_value_mean_by_col(df, 'cartão', x))
df.loc[:, "total_uso_cartao"] = df.loc[:, "cartão"].apply(lambda x: get_count(df, 'cartão', x))

df_final = df.loc[:, ['dia','dia_semana', 'periodo_semana', 'periodo_mes', 'hora', 'hora_completa_segundos', 'periodo_dia', 'cartão', 'valor', 'media_dia', 'media_cartao', 'total_uso_cartao', 'cbk']] 
df_final.drop_duplicates(inplace=True)

In [11]:
uniq_cards = df_final['cartão'].drop_duplicates().values
df_final['vez_uso_cartao'] = 0
for card in uniq_cards:
    df_final.loc[df_final['cartão'] == card, 'vez_uso_cartao'] = [x + 1 for x in range(len(df_final[df_final['cartão'] == card]))]

df_final.loc[:, "cartao_usado_antes"] = df_final.loc[:, "vez_uso_cartao"].apply(lambda x: x > 1)

In [12]:
df_final

,dia,dia_semana,periodo_semana,periodo_mes,hora,hora_completa_segundos,periodo_dia,cartão,valor,media_dia,media_cartao,total_uso_cartao,cbk,vez_uso_cartao,cartao_usado_antes
0,1,4,semana,inicio,0,114,madrugada,536518******2108,36.54,139.59,36.54,2,Não,1,False
1,1,4,semana,inicio,0,226,madrugada,536518******2108,36.54,139.59,36.54,2,Não,2,True
2,1,4,semana,inicio,0,530,madrugada,453211******1239,69.00,139.59,69.00,1,Não,1,False
3,1,4,semana,inicio,0,1620,madrugada,548827******1705,193.43,139.59,193.43,1,Não,1,False
4,1,4,semana,inicio,1,5566,madrugada,531681******9778,132.00,139.59,132.00,1,Não,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11000,30,5,fds,fim,23,83327,noite,439354******5281,15.00,92.96,15.00,2,Não,1,False
11001,30,5,fds,fim,23,83724,noite,549167******1648,20.00,92.96,20.00,2,Não,1,False
11002,30,5,fds,fim,23,83861,noite,518759******8384,70.00,92.96,70.00,2,Não,1,False
11003,30,5,fds,fim,23,85891,noite,518759******0329,20.00,92.96,20.00,2,Não,1,False


In [13]:
df_final.to_csv("../data/chargeback_data_final.csv", index=False)